<a href="https://colab.research.google.com/github/Dhawal-Modi/DSC_2024_Space_Jam/blob/nikhil/Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import sys
sys.path.append("/content/drive/MyDrive/task3")

In [5]:
import torch
from torchsummary import summary
class FireModule(torch.nn.Module):
    def __init__(self, in_channels, squeeze_channels, expand1x1_channels, expand1x3_channels):
        super(FireModule, self).__init__()
        self.squeeze = torch.nn.Conv1d(in_channels, squeeze_channels, kernel_size=1)
        self.relu = torch.nn.ReLU(inplace=True)
        self.expand1x1 = torch.nn.Conv1d(squeeze_channels, expand1x1_channels, kernel_size=1)
        self.expand1x3 = torch.nn.Conv1d(squeeze_channels, expand1x3_channels, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.squeeze(x)
        x = self.relu(x)
        out1x1 = self.expand1x1(x)
        out1x3 = self.expand1x3(x)
        out = torch.cat([out1x1, out1x3], dim=1)
        return self.relu(out)


class SqueezeNet(torch.nn.Module):
    def __init__(self,in_channels,classes):
        super(SqueezeNet, self).__init__()
        self.features = torch.nn.Sequential(
            # conv1
            torch.nn.Conv1d(in_channels, 64, kernel_size=3, stride=2, padding=1),
            torch.nn.ReLU(inplace=True),
            # maxpool1
            torch.nn.MaxPool1d(kernel_size=3, stride=2, padding=1),
            # Fire2
            FireModule(64, 16, 64, 64),
            # Fire3
            FireModule(128, 16, 64, 64),
            # maxpool4
            torch.nn.MaxPool1d(kernel_size=3, stride=2, padding=1),
            # Fire5
            FireModule(128, 32, 128, 128),
            # Fire6
            FireModule(256, 32, 128, 128),
            #maxpool7
            torch.nn.MaxPool1d(kernel_size=3, stride=2, padding=1),
            # Fire8
            FireModule(256, 48, 192, 192),
            # Fire9
            FireModule(384, 48, 192, 192),
            #Fire10
            FireModule(384, 64, 256, 256),
            # Fire11
            FireModule(512, 64, 256, 256),
            #dropout12
            torch.nn.Dropout(p=0.1, inplace=False),
        )

        self.classifier = torch.nn.Sequential(
            torch.nn.Conv1d(512, 75, kernel_size=3, stride=2, padding=0),
            torch.nn.AdaptiveAvgPool1d(1),
            torch.nn.Conv1d(75, 75, kernel_size=3, stride=2, padding=0)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return x

Number of file pairs: 0
Example of file pair:


IndexError: list index out of range